TODO: na grafoch ukazat pocet jednotlivich unikatnych hodnot pre kazdu categorical hodnotu

In [1]:
import pandas as pd
import json

from sklearn.preprocessing import LabelEncoder, StandardScaler
# import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, concatenate, Dense, InputLayer
from tensorflow.keras.models import Sequential, Model

In [2]:
with open('../../../config.json') as config_file:
    config = json.load(config_file)

context_data_path = config['context_data']

In [3]:
context_data = pd.read_csv('../../..'+context_data_path)
context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,holiday,user_gender,user_age,user_occupation,user_zip_code,movie_year
0,1,1193,7,0,4,4,new_years,F,1,10,48067,1975
1,1,661,7,0,4,4,new_years,F,1,10,48067,1996
2,1,914,7,0,4,4,new_years,F,1,10,48067,1964
3,1,3408,7,0,4,4,new_years,F,1,10,48067,2000
4,1,2355,7,0,4,4,no_holiday,F,1,10,48067,1998
...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,3,1,1,4,no_holiday,M,25,6,11106,1989
1000205,6040,1094,3,1,1,4,no_holiday,M,25,6,11106,1992
1000206,6040,562,3,1,1,4,no_holiday,M,25,6,11106,1995
1000207,6040,1096,3,1,1,4,no_holiday,M,25,6,11106,1982


In [4]:
# Using one-hot encoding for columns 'holiday' and 'user_gender'
context_data = pd.get_dummies(context_data, columns=['holiday', 'user_gender'])
context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,user_age,user_occupation,user_zip_code,movie_year,holiday_christmas,holiday_new_years,holiday_no_holiday,holiday_summer_holiday,holiday_thanksgiving,holiday_valentines,holiday_veterans_day,user_gender_F,user_gender_M
0,1,1193,7,0,4,4,1,10,48067,1975,False,True,False,False,False,False,False,True,False
1,1,661,7,0,4,4,1,10,48067,1996,False,True,False,False,False,False,False,True,False
2,1,914,7,0,4,4,1,10,48067,1964,False,True,False,False,False,False,False,True,False
3,1,3408,7,0,4,4,1,10,48067,2000,False,True,False,False,False,False,False,True,False
4,1,2355,7,0,4,4,1,10,48067,1998,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,3,1,1,4,25,6,11106,1989,False,False,True,False,False,False,False,False,True
1000205,6040,1094,3,1,1,4,25,6,11106,1992,False,False,True,False,False,False,False,False,True
1000206,6040,562,3,1,1,4,25,6,11106,1995,False,False,True,False,False,False,False,False,True
1000207,6040,1096,3,1,1,4,25,6,11106,1982,False,False,True,False,False,False,False,False,True


In [5]:
# Label encode other categorical columns
label_encoders = {}
categorical_columns = ['user_id', 'movie_id', 'day', 'is_weekday', 'season', 'part_of_day', 'user_age', 'user_occupation']
for column in categorical_columns:
    encoder = LabelEncoder()
    context_data[column] = encoder.fit_transform(context_data[column])
    label_encoders[column] = encoder
context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,user_age,user_occupation,user_zip_code,movie_year,holiday_christmas,holiday_new_years,holiday_no_holiday,holiday_summer_holiday,holiday_thanksgiving,holiday_valentines,holiday_veterans_day,user_gender_F,user_gender_M
0,0,1104,6,0,3,3,0,10,48067,1975,False,True,False,False,False,False,False,True,False
1,0,639,6,0,3,3,0,10,48067,1996,False,True,False,False,False,False,False,True,False
2,0,853,6,0,3,3,0,10,48067,1964,False,True,False,False,False,False,False,True,False
3,0,3177,6,0,3,3,0,10,48067,2000,False,True,False,False,False,False,False,True,False
4,0,2162,6,0,3,3,0,10,48067,1998,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6039,1019,2,1,0,3,2,6,11106,1989,False,False,True,False,False,False,False,False,True
1000205,6039,1022,2,1,0,3,2,6,11106,1992,False,False,True,False,False,False,False,False,True
1000206,6039,548,2,1,0,3,2,6,11106,1995,False,False,True,False,False,False,False,False,True
1000207,6039,1024,2,1,0,3,2,6,11106,1982,False,False,True,False,False,False,False,False,True


In [6]:
# Standardize numerical columns
numerical_columns = ['user_zip_code', 'movie_year']
scaler = StandardScaler()
context_data[numerical_columns] = scaler.fit_transform(context_data[numerical_columns])
context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,user_age,user_occupation,user_zip_code,movie_year,holiday_christmas,holiday_new_years,holiday_no_holiday,holiday_summer_holiday,holiday_thanksgiving,holiday_valentines,holiday_veterans_day,user_gender_F,user_gender_M
0,0,1104,6,0,3,3,0,10,-0.192078,-0.815259,False,True,False,False,False,False,False,True,False
1,0,639,6,0,3,3,0,10,-0.192078,0.648225,False,True,False,False,False,False,False,True,False
2,0,853,6,0,3,3,0,10,-0.192078,-1.581845,False,True,False,False,False,False,False,True,False
3,0,3177,6,0,3,3,0,10,-0.192078,0.926984,False,True,False,False,False,False,False,True,False
4,0,2162,6,0,3,3,0,10,-0.192078,0.787605,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6039,1019,2,1,0,3,2,6,-1.343850,0.160397,False,False,True,False,False,False,False,False,True
1000205,6039,1022,2,1,0,3,2,6,-1.343850,0.369466,False,False,True,False,False,False,False,False,True
1000206,6039,548,2,1,0,3,2,6,-1.343850,0.578536,False,False,True,False,False,False,False,False,True
1000207,6039,1024,2,1,0,3,2,6,-1.343850,-0.327431,False,False,True,False,False,False,False,False,True


In [7]:
numerical_inputs = []

for col in context_data.columns:
    if col == 'user_zip_code' or col == 'movie_year':
        input_layer = Input(shape=(1,), name=col)
        numerical_inputs.append(input_layer)
numerical_inputs

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'user_zip_code')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'movie_year')>]

In [8]:
# Define embedding input sizes and dimensions
embedding_input_sizes = {
    'user_id': len(label_encoders['user_id'].classes_),
    'movie_id': len(label_encoders['movie_id'].classes_),
    'day': len(label_encoders['day'].classes_),
    'is_weekday': len(label_encoders['is_weekday'].classes_),
    'season': len(label_encoders['season'].classes_),
    'part_of_day': len(label_encoders['part_of_day'].classes_),
    'user_age': len(label_encoders['user_age'].classes_),
    'user_occupation': len(label_encoders['user_occupation'].classes_),
}
embedding_input_sizes

{'user_id': 6040,
 'movie_id': 3706,
 'day': 7,
 'is_weekday': 2,
 'season': 4,
 'part_of_day': 4,
 'user_age': 7,
 'user_occupation': 21}

user_id and movie_id is going to be set with 16 dimensions (embedding) in the model - this can be changed as hyperparameter later

In [9]:
embedding_dimensions = {
    'user_id': 16,
    'movie_id': 16,    
    'day': 8,
    'is_weekday': 2,
    'season': 8,
    'part_of_day': 8,
    'user_age': 8,
    'user_occupation': 16, # user_occupation has 21 categorias
}

In [10]:
# Define the input layers for embeddings
embedding_inputs = {}
for col in embedding_input_sizes.keys():
    embedding_inputs[col] = Input(shape=(1,), name=f'{col}_input')
embedding_inputs

{'user_id': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'user_id_input')>,
 'movie_id': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'movie_id_input')>,
 'day': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'day_input')>,
 'is_weekday': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'is_weekday_input')>,
 'season': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'season_input')>,
 'part_of_day': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'part_of_day_input')>,
 'user_age': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'user_age_input')>,
 'user_occupation': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'user_occupation_input')>}

In [11]:
# Create embedding layers
embedding_layers = {}
for col in embedding_input_sizes.keys():
    embedding_layers[col] = Embedding(
        input_dim=embedding_input_sizes[col],
        output_dim=embedding_dimensions[col],
        input_length=1,
        name=f'{col}_embedding'
    )(embedding_inputs[col])
embedding_layers

{'user_id': <KerasTensor: shape=(None, 1, 16) dtype=float32 (created by layer 'user_id_embedding')>,
 'movie_id': <KerasTensor: shape=(None, 1, 16) dtype=float32 (created by layer 'movie_id_embedding')>,
 'day': <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'day_embedding')>,
 'is_weekday': <KerasTensor: shape=(None, 1, 2) dtype=float32 (created by layer 'is_weekday_embedding')>,
 'season': <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'season_embedding')>,
 'part_of_day': <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'part_of_day_embedding')>,
 'user_age': <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'user_age_embedding')>,
 'user_occupation': <KerasTensor: shape=(None, 1, 16) dtype=float32 (created by layer 'user_occupation_embedding')>}

In [12]:
# Flatten embedding layers
flattened_embeddings = []

for col in embedding_input_sizes.keys():
    embedding_layer = embedding_layers[col]
    flattened_layer = Flatten(name=f'flattened_{col}')(embedding_layer)
    flattened_embeddings.append(flattened_layer)
flattened_embeddings

[<KerasTensor: shape=(None, 16) dtype=float32 (created by layer 'flattened_user_id')>,
 <KerasTensor: shape=(None, 16) dtype=float32 (created by layer 'flattened_movie_id')>,
 <KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'flattened_day')>,
 <KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'flattened_is_weekday')>,
 <KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'flattened_season')>,
 <KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'flattened_part_of_day')>,
 <KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'flattened_user_age')>,
 <KerasTensor: shape=(None, 16) dtype=float32 (created by layer 'flattened_user_occupation')>]

In [13]:
one_hot_encoded_inputs = []

for col in context_data.columns:
    if col.startswith('holiday_') or col.startswith('user_gender_'):
        input_layer = Input(shape=(1,), name=col)
        one_hot_encoded_inputs.append(input_layer)
        
one_hot_encoded_inputs

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'holiday_christmas')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'holiday_new_years')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'holiday_no_holiday')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'holiday_summer_holiday')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'holiday_thanksgiving')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'holiday_valentines')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'holiday_veterans_day')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'user_gender_F')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'user_gender_M')>]

In [14]:
merged_embeddings = concatenate(flattened_embeddings + one_hot_encoded_inputs + numerical_inputs, name='emmbedings_concat')
merged_embeddings

<KerasTensor: shape=(None, 93) dtype=float32 (created by layer 'emmbedings_concat')>

In [15]:
model = Sequential()

model.add(Input(shape=(merged_embeddings.shape[1],), tensor=merged_embeddings))

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

In [16]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                6016      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 8129 (31.75 KB)
Trainable params: 8129 (31.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
